# Train
training model

In [2]:
from train import initialize_globals, Hyperparameters, pipline

initialize_globals()
hyperparameters = Hyperparameters(
    epochs=50,
    early_stop_patience=5,
    lr_warmup_epochs=5,
    lr=0.0001
)

hyperparameters

mps is available device


Hyperparameters(model='stn_mobilenet_small', val_ratio=0.2, data_ratio=1, batch_size=64, epochs=50, early_stop_patience=30, target_val_acc=100.0, opt_name='sgd', momentum=0.9, lr=0.0001, lr_scheduler_name='cosine', lr_warmup_epochs=10, lr_warmup_method='linear', lr_warmup_decay=0.01, weight_decay=2e-05, norm_weight_decay=0.0, label_smoothing=0.1, train_crop_size=176, val_crop_size=224)

In [ ]:
pipline(hyperparameters)